Задание 1: Скачать нейросети ResNet и написать короткую процедуру для предсказания класса изображения

In [1]:
import torch
from torchvision import models, transforms
from PIL import Image

In [2]:
resnet = models.resnet50(pretrained=True)
resnet.eval()  # Перевести модель в режим оценки

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img = Image.open("/content/corgi.jpg")  # Путь к изображению
img_t = preprocess(img)
batch_t = torch.unsqueeze(img_t, 0)

In [4]:
with torch.no_grad():
    out = resnet(batch_t)
_, indices = torch.max(out, 1)

# Загрузка классов ImageNet
with open('/content/imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

# Получение предсказанного класса
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(classes[indices[0]], percentage[indices[0]].item())

263: 'Pembroke, Pembroke Welsh corgi', 84.73977661132812


Задание 2: Скачать нейросеть BERT для лингвистических задач и реализовать процедуру классификации текстов (без оглядки на качество классификации)

In [5]:
!pip install transformers

In [6]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Предположим, что есть набор данных в следующем формате:
texts = [
    "Это фантастический продукт, который я очень рекомендую всем!",
    "Я очень разочарован этой услугой, она совершенно не оправдала моих ожиданий."
]
labels = [1, 0]  # 1 для положительных отзывов, 0 для отрицательных

# Разделение на тренировочную и тестовую выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

# Определение пользовательского класса Dataset для работы с DataLoader
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Загрузка предварительно обученного токенизатора и модели BERT для русского языка
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=2)

# Создание DataLoader для тренировочной и тестовой выборок
train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len=512)
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_len=512)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

# Определение оптимизатора
optimizer = AdamW(model.parameters(), lr=2e-5)

# Перемещение модели на GPU, если доступен
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Функция для тренировки модели
def train_epoch(model, data_loader, optimizer, device, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs[0]
        logits = outputs[1]
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

# Функция для оценки модели
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs[0]
            logits = outputs[1]
            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

# Тренировка модели
for epoch in range(4):
    print(f'Epoch {epoch + 1}/4')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        optimizer,
        device,
        len(train_dataset)
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        test_loader,
        device,
        len(test_dataset)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()

# Сохранение модели
model.save_pretrained("my_bert_model")
tokenizer.save_pretrained("my_bert_model")

# Загрузка модели для инференса
model = BertForSequenceClassification.from_pretrained("my_bert_model")
tokenizer = BertTokenizer.from_pretrained("my_bert_model")

# Пример инференса
text = "Разочарован в продукте."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
prediction = torch.argmax(output.logits, dim=-1)
print(f'Predicted class: {prediction.item()}')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use 

Epoch 1/4
----------
Train loss 0.5994464755058289 accuracy 1.0
Val   loss 0.8659203052520752 accuracy 0.0

Epoch 2/4
----------
Train loss 0.5481449365615845 accuracy 1.0
Val   loss 1.0051543712615967 accuracy 0.0

Epoch 3/4
----------
Train loss 0.4958711266517639 accuracy 1.0
Val   loss 1.1356463432312012 accuracy 0.0

Epoch 4/4
----------
Train loss 0.5078164339065552 accuracy 1.0
Val   loss 1.3819769620895386 accuracy 0.0

Predicted class: 0
